Local imports and initialization

In [8]:
import yaml
import sys, os

from IPython.display import Image, SVG

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from src.utils import *
from src.svg_quality_checks import *
from src.gpt_wrappers import *

# show svg 
def show_svg(file):
    display(SVG(file))

# load yaml config
with open("../config.yml", "r") as f:
    config = yaml.safe_load(f)


OpenAI and wandb related imports and initialization

In [9]:
import openai
import wandb

openai.api_key = config["OPENAI_KEY"]

run = wandb.init(
    project=config['WANDB_PROJECT'], 
    entity=config['WANDB_ENTITY']
)

results_table = wandb.Table(
    columns=[   "match",
                "system_prompt", 
                "question_prompt",
                "input_css",
                "output_css",
                "original_css",
    ]
)

# Zero-shot learning (no examples)

In [10]:
system_prompt = """
You know that CSS is a language for describing the rendering of structured documents such as HTML and XML.
You know that Flexible Box Layout is an specification that describes a CSS box model optimized for user interface design. 
In the flex layout model, the children of a flex container can be laid out in any direction, and can “flex” their sizes, 
either growing to fill unused space or shrinking to avoid overflowing the parent. 
Both horizontal and vertical alignment of the children can be easily manipulated. 
Nesting of these boxes (horizontal inside vertical, or vertical inside horizontal) can be used to build layouts in two dimensions.

You are an expert in user interface design and based on an initial and incomplete CSS flexbox text input, you are always able to generate the most probable missing part.
"""

Case 0: take a fragment of CSS and generate the rest

In [11]:
FILENAME = "board2"
css_incomplete = load_css(FILENAME)
css_original = load_css(FILENAME, css_path = '../data/css/original')

In [12]:
question = f"""

    This is an incomplete CSS text that uses Flexible Box Layout: \n
  {css_incomplete}
      Take the above input, by looking at its Flexible Box Layout properties and attribues, figure out what is the single most likely Flexible Box Layout property to follow and append it to the original input create a valid CSS output.
"""

chatgpt = VanillaChatGPTWrapper(system_prompt)
answer = chatgpt.generate(question)
save_autocomplete_css(answer, FILENAME)
match = compare_css(answer, css_original)
print(answer)


['  /* missing property */', '  align-items: start;', '  background: rgba(24, 71, 212, 1);', '  display: flex;', '  flex-direction: row;', '  flex-shrink: 1;', '  flex-wrap: nowrap;', '  gap: 10px;', '  height: auto;', '  justify-content: space-around;', '  left: 106px;', '  padding: 0px;', '  top: 83px;', '  width: auto;', '.board {', '/* Board */', '}']
['  align-items: start;', '  background: rgba(24, 71, 212, 1);', '  display: flex;', '  flex-direction: row;', '  flex-wrap: nowrap;', '  gap: 10px;', '  height: auto;', '  justify-content: space-around;', '  left: 106px;', '  padding: 0px;', '  position: relative;', '  top: 83px;', '  width: auto;', '.board {', '/* Board */', '}']
/* Board */
.board {
  width: auto;
  height: auto;
  left: 106px;
  top: 83px;
  background: rgba(24, 71, 212, 1);
  display: flex;
  flex-direction: row;
  align-items: start;
  justify-content: space-around;
  gap: 10px;
  padding: 0px;
  flex-wrap: nowrap;
  /* missing property */
  flex-shrink: 1;
}


In [13]:
results_table.add_data(
    *[
        match, system_prompt, question,
        css_incomplete, answer, css_original
    ]
)

Publish results to wandb

In [14]:
run.log({"results_table": results_table})